In [24]:
## creating dataframe
import pandas as pd

db=pd.read_excel('DB.xlsx')
# Create DataFrame
df = db

# Add a new column for suspect price
df["suspect_price"] = None

# Initialize suspect price
suspect_price = None

# Iterate through each row
for index, row in df.iterrows():
    if row["transaction_type"] == "suspect":
        suspect_price = row["price"]  # Set suspect price
    df.at[index, "suspect_price"] = suspect_price  # Add suspect price to the row

#delete added column as index
df.reset_index(drop=True, inplace=True)

# Drop rows where suspect_price is None
df.dropna(subset=["suspect_price"], inplace=True)
# Print the updated DataFrame
df.to_excel('labeled_db.xlsx')

In [1]:
## creatinf df without suspect rows
no_sus_df = df[df['transaction_type'] != 'suspect']
no_sus_df.to_excel('no_sus_df.xlsx')

NameError: name 'df' is not defined

In [2]:
import pandas as pd
df = pd.read_excel('no_sus_df.xlsx')

In [75]:
###  1 by1 tagged input
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

df = pd.read_excel('no_sus_df.xlsx')

# Sample data (assuming df is your DataFrame)
data = df

# Convert 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Create DataFrame
df = pd.DataFrame(data)

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Define features (X) and target (y) for train and test sets
X_train = train_df[['price', 'transaction_type']]
y_train = train_df['suspect_price']
X_test = test_df[['price', 'transaction_type']]
y_test = test_df['suspect_price']

# Define preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('transaction', OneHotEncoder(), ['transaction_type'])
    ],
    remainder='passthrough'
)

# Create pipeline with preprocessing and model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)



Mean Squared Error: 10143.97483051181


In [76]:
##evaluation previous code

# Convert the NumPy array to a pandas DataFrame for y_pred
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted_Suspect_Price'])

# Convert y_test Series to DataFrame and reset index
y_test_df = y_test.to_frame().reset_index(drop=True)

# Concatenate X_test, y_pred_df, and y_test_df
result_df = pd.concat([X_test.reset_index(drop=True), y_pred_df, y_test_df], axis=1)

# Rename the y_test column to Actual_Suspect_Price for clarity
result_df.rename(columns={'suspect_price': 'Actual_Suspect_Price'}, inplace=True)

# Save the DataFrame to an Excel file
result_df.to_excel('predicted_results_1by1_tagged.xlsx', index=False)


In [66]:
## arima model

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

df = pd.read_excel('no_sus_df.xlsx')

df['time'] = pd.to_datetime(df['time'])

df.set_index('time', inplace=True)

# Extract the time series (price)
price_series = df['price']

# Fit the ARIMA model
model = ARIMA(price_series, order=(1, 0, 1))  # Example order (p, d, q)
fit_model = model.fit()

# Forecast the next suspect_price
forecast = fit_model.forecast(steps=1)



C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting p

In [29]:
### one by one no tag input
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_excel('no_sus_df.xlsx')

# Sample data (assuming df is your DataFrame)
data = df

# Convert 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Create DataFrame
df = pd.DataFrame(data)

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Define features (X) and target (y) for train and test sets
X_train = train_df[['price']]
y_train = train_df['suspect_price']
X_test = test_df[['price']]
y_test = test_df['suspect_price']

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

##evaluation

# Convert the NumPy array to a pandas DataFrame for y_pred
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted_Suspect_Price'])

# Convert y_test Series to DataFrame and reset index
y_test_df = y_test.to_frame().reset_index(drop=True)

# Concatenate X_test, y_pred_df, and y_test_df
result_df = pd.concat([X_test.reset_index(drop=True), y_pred_df, y_test_df], axis=1)

# Rename the y_test column to Actual_Suspect_Price for clarity
result_df.rename(columns={'suspect_price': 'Actual_Suspect_Price'}, inplace=True)

# Save the DataFrame to an Excel file
result_df.to_excel('predicted_results.xlsx', index=False)




Predicted Suspect Price for New Price Value: 15025.3637240175


C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [95]:
### one by one no tag , 2 feature input
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

df = pd.read_excel('no_sus_df.xlsx')

# Sample data (assuming df is your DataFrame)
data = df

# Convert 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Create DataFrame
df = pd.DataFrame(data)

# Define a function to map transaction types to numerical values
def map_transaction_code(transaction_type):
    if transaction_type == 'sell':
        return 3
    elif transaction_type == 'buy':
        return -3
    elif transaction_type == 'deal':
        return 1
    else:
        return 0  # Handle other cases if necessary

# Add a new column 'transaction_code' with numerical representation
df['transaction_code'] = df['transaction_type'].apply(map_transaction_code)

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Define features (X) and target (y) for train and test sets
X_train = train_df[['price','transaction_code']]
y_train = train_df['suspect_price']
X_test = test_df[['price','transaction_code']]
y_test = test_df['suspect_price']

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

##evaluation

# Convert the NumPy array to a pandas DataFrame for y_pred
y_pred_df = pd.DataFrame(y_pred, columns=['Predicted_Suspect_Price'])

# Convert y_test Series to DataFrame and reset index
y_test_df = y_test.to_frame().reset_index(drop=True)

# Concatenate X_test, y_pred_df, and y_test_df
result_df = pd.concat([X_test.reset_index(drop=True), y_pred_df, y_test_df], axis=1)

# Rename the y_test column to Actual_Suspect_Price for clarity
result_df.rename(columns={'suspect_price': 'Actual_Suspect_Price'}, inplace=True)

# Save the DataFrame to an Excel file
result_df.to_excel('predicted_results_notag_2features.xlsx', index=False)




In [1]:
### one by one no tag , 2 feature input [random forest]

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Sample data (assuming df is your DataFrame)
data = df

# Convert 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Create DataFrame
df = pd.DataFrame(data)

# Split the data into train and test sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

# Define features (X) and target (y) for train and test sets
X_train = train_df[['price', 'transaction_type']]
y_train = train_df['suspect_price']
X_test = test_df[['price', 'transaction_type']]
y_test = test_df['suspect_price']

# Initialize and train the Random Forest classifier
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

# You can use the trained model for further predictions or analysis


In [1]:
import xgboost as xgb

In [61]:
### 10 by 10 no tag input

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

df = pd.read_excel('no_sus_df.xlsx')

# Sample data (assuming df is your DataFrame)
data = df

# Convert 'time' column to datetime
data['time'] = pd.to_datetime(data['time'])

# Create DataFrame
df = pd.DataFrame(data)

# Define the sequence length
sequence_length = 10

# Initialize lists to store sequences and corresponding labels
sequences = []
labels = []

# Create sequences of prices and corresponding labels
for i in range(len(df) - sequence_length + 1):
    sequences.append(df['price'].iloc[i:i+sequence_length])
    labels.append(df['suspect_price'].iloc[i+sequence_length-1])

# Convert lists to arrays
sequences = np.array(sequences)
labels = np.array(labels)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(sequences, labels, test_size=0.2, shuffle=False)

# Standardize the input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the input sequences for compatibility with LSTM
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Define the LSTM model
model = Sequential([
    LSTM(50, input_shape=(sequence_length, 1)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# Predict on the test set
y_pred = model.predict(X_test)


# Predict the suspect_price for new sequences
# Example: assuming you have a new sequence of prices stored in a variable called new_sequence
new_sequence = np.array([[15020, 15020, 15015, 15015, 15015, 15020, 15020, 15015, 15020, 15015]])  # Example sequence of prices
new_sequence = scaler.transform(new_sequence)
new_sequence = new_sequence.reshape(1, sequence_length, 1)
new_pred = model.predict(new_sequence)
print('Predicted Suspect Price for New Sequence of Prices:', new_pred[0][0])


Epoch 1/50


C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


52/52 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 221822560.0000
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 221654000.0000
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 221391168.0000
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 221259888.0000
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 221130976.0000
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 221117632.0000
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 220984496.0000
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 220914384.0000
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 220781760.0000
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 220780816.0000
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 220667152.0000
Epoch 12/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 220634032.0000
Epoch 13/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 220543536.0000
Epoch 14/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss:

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

dff = pd.read_excel('no_sus_df.xlsx') 

# Select the 100 newest rows
df = dff.tail(100)


# Select the 10 newest rows
newest_rows = df.tail(10)

# Convert 'time' column to datetime
newest_rows['time'] = pd.to_datetime(newest_rows['time'])

# Define the sequence length
sequence_length = 10

# Initialize lists to store sequences and corresponding labels
sequences = []
labels = []

# Create sequences of prices and corresponding labels
for i in range(len(newest_rows) - sequence_length + 1):
    sequences.append(newest_rows['price'].iloc[i:i+sequence_length])
    labels.append(newest_rows['suspect_price'].iloc[i+sequence_length-1])

# Convert lists to arrays
sequences = np.array(sequences)
labels = np.array(labels)

# Standardize the input features
scaler = StandardScaler()
sequences_scaled = scaler.fit_transform(sequences)

# Reshape the input sequences for compatibility with LSTM
sequences_reshaped = sequences_scaled.reshape(sequences_scaled.shape[0], sequences_scaled.shape[1], 1)

# Define the LSTM model
model = Sequential([
    LSTM(50, input_shape=(sequence_length, 1)),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(sequences_reshaped, labels, epochs=50, batch_size=32, verbose=1)

# Predict on the most recent sequence
new_sequence = newest_rows['price'].values.reshape(1, sequence_length)  # Remove the extra dimension
new_sequence_scaled = scaler.transform(new_sequence)
new_sequence_scaled = new_sequence_scaled.reshape(1, sequence_length, 1)  # Reshape back to 3D for LSTM input
new_pred = model.predict(new_sequence_scaled)
print('Predicted Suspect Price for New Sequence of Prices:', new_pred[0][0])


Epoch 1/50


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4060\3404299695.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newest_rows['time'] = pd.to_datetime(newest_rows['time'])
C:\Users\Lenovo\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 219928896.0000
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 219928608.0000
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 219928320.0000
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 219927968.0000
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 219927632.0000
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 219927216.0000
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 219926784.0000
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 219926320.0000
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 219925808.0000
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 219925216.0000
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 219924592.0000
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 219923856.0000
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 219923056.0000
Epoch 14/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 219922128.0000
